<a href="https://colab.research.google.com/github/jessicasalazar/Optimization/blob/main/Location_models_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
  <img src="https://www.eafit.edu.co/institucional/sostenibilidad-ambiental/PublishingImages/ecologia-urbana/logos/eafit-logo.png" alt="Logo EAFIT" width="150">
</div>

# Comparative analysis between the Covering Location Problem and the Maximal Covering Location Problem

#### Nombre: Jessica Salazar Vásquez
#### Profesor: Juan Carlos Duque

In [1]:
# To install libraries
!pip install pyomo
!apt install glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [6]:

# To import libraries
from pyomo.environ import *
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, Constraint, Objective, maximize,
    NonNegativeReals, NonNegativeIntegers, SolverFactory
)

# PAPER: THE LOCATION OF EMERGENCY SERVICE FACILITIES (ReVelle et al., 1971)
---
### The Covering Location Problem
---

In [4]:
# ----------------------
# Set of nodes (demands)
# ----------------------
demands = [0, 1, 2, 3, 4]

# -----------------------------------------------------------------------
# Set of possible allocation points
# (This example has the same set of nodes and set of possible allocations)
# -----------------------------------------------------------------------
facilities = [0, 1, 2, 3, 4]

# -------------------------------------------
# Dictionary: for each of demand i,
# N[i] = list of nodes j to covering i
# -------------------------------------------
N = {
    0: [0, 1, 4],
    1: [0, 1, 2],
    2: [1, 2, 3],
    3: [2, 3, 4],
    4: [0, 3, 4]
}

# To create the model
model = ConcreteModel(name="EmergencyServiceFacilities") # Class used to define concrete optimization model in Pyomo

# To define binary variables x_j
model.x = Var(facilities, domain=Binary)

# Objective function: minimize the total number of facilities
def obj_rule(m):
    return sum(m.x[j] for j in facilities)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Covering resticted:
# Each node i must be covered by at least one facility in N[i]
def cover_rule(m, i):
    return sum(m.x[j] for j in N[i]) >= 1

model.cover = Constraint(demands, rule=cover_rule)

# The solver is GLPK is the default solver
# GNU Linear Programming Kit, used to solve Linear Programming (LP) and Integer Programming (IP)
solver = SolverFactory('glpk')
results = solver.solve(model)

# Show the state of the solution
print("State of the solution:", results.solver.status)
print("Termination condition of the solver:", results.solver.termination_condition)

# Show optimal value of the objective function
print("Optimal value (number of facilities):", model.obj())

# Show which facilities open
for j in facilities:
    print(f"x[{j}] =", model.x[j].value)

# Complete report:
model.display()

State of the solution: ok
Termination condition of the solver: optimal
Optimal value (number of facilities): 2.0
x[0] = 1.0
x[1] = 0.0
x[2] = 1.0
x[3] = 0.0
x[4] = 0.0
Model EmergencyServiceFacilities

  Variables:
    x : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   2.0

  Constraints:
    cover : Size=5
        Key : Lower : Body : Upper
          0 :   1.0 :  1.0 :  None
          1 :   1.0 :  2.0 :  None
          2 :   1.0 :  1.0 :  None
          3 :   1.0 :  1.0 :  None
          4 :   1.0 :  1.0 :  None


# Modification 1: Set selection penalty by JessicaSalazar

A subset with a high penalty value will be more costly to open, and the solver will try not to select it unless it is necessary to cover the demand.

This can mean that some subsets might be saturated and therefore difficult to service, so we penalize them in order not to prioritize them in the solution unless necessary.

In [5]:
# --------------
# Sets
# --------------
demands = [0, 1, 2, 3, 4]
facilities = [0, 1, 2, 3, 4]

# -------------------------------------------
# Dictionary: for each of demand i,
# N[i] = list of nodes j to covering i
# -------------------------------------------
N = {
    0: [0, 1, 4],
    1: [0, 1, 2],
    2: [1, 2, 3],
    3: [2, 3, 4],
    4: [0, 3, 4]
}

# ----------------------------------------------
# I define a cost of c[j] for each node j
# (I can modify it to review the changes)
# ----------------------------------------------
penalty = {
    0: 1,   # Open the facility 0, it cost 1
    1: 1,   # Open the facility 1, it cost 1
    2: 4,   # Open the facility 2, it cost 1
    3: 1,   # Open the facility 3, it cost 5 (selection penalty)
    4: 2    # Open the facility 4, it cost 2
}

model = ConcreteModel(name="EmergencyServiceFacilities")

# To define binary variables x_j
model.x = Var(facilities, domain=Binary)

# Objective function: minimize the total costs of facilities
def obj_rule(m):
    return sum(penalty[j] * m.x[j] for j in facilities)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Covering resticted:
# Each node i must be covered by at least one facility in N[i]
def cover_rule(m, i):
    return sum(m.x[j] for j in N[i]) >= 1

model.cover = Constraint(demands, rule=cover_rule)

# Solver
solver = SolverFactory('glpk')
results = solver.solve(model, tee=False)

print("State of the solution:", results.solver.status)
print("Termination condition of the solver:", results.solver.termination_condition)
print("Optimal value (number of facilities):", model.obj())

for j in facilities:
    print(f"x[{j}] =", model.x[j].value)

State of the solution: ok
Termination condition of the solver: optimal
Optimal value (number of facilities): 2.0
x[0] = 1.0
x[1] = 0.0
x[2] = 0.0
x[3] = 1.0
x[4] = 0.0


# PAPER: THE MAXIMAL COVERING LOCATION PROBLEM (Church et al. 1974)
---
### The Maximal Covering Location Problem
---

In [ ]:
# ----------------------
# Set of demand nodes
# ----------------------
I_data = [1, 2, 3, 4, 5]

# ----------------------
# Set of facility sites
# ----------------------
J_data = [1, 2, 3]

# -------------------------------------------
# Dictionary: for each of demand i,
# N[i] = list of nodes j to covering i
# -------------------------------------------
N_i_data = {
    1: [1, 2],
    2: [1],
    3: [2, 3],
    4: [2],
    5: [1, 3]
}

# -------------------------------------------
# Dictionary: for each of demand importance.
# -------------------------------------------
w_data = {
    1: 100.0,
    2: 50.0,
    3: 80.0,
    4: 60.0,
    5: 120.0
}
# -----------------------------
# Minimun number of facilities
# -----------------------------
p_value = 1

# -----------------------------------------------------
# Create model
# The model uded is Maximal Covering Location Problem
# -----------------------------------------------------
model = ConcreteModel(name="MCLP")

# Sets
model.I = Set(initialize=I_data)
model.J = Set(initialize=J_data)

# Parameters
model.w = Param(model.I, initialize=w_data, within=NonNegativeReals)
model.p = Param(initialize=p_value, within=NonNegativeIntegers)

# Variables
model.x = Var(model.J, within=Binary)
model.y = Var(model.I, within=Binary)

# Covering restricted:
# Each node i must be covered by at least one facility in N[i]
def coverage_rule(model, i):
    return model.y[i] + sum(model.x[j] for j in N_i_data[i]) >= 1
model.CoverageConstraint = Constraint(model.I, rule=coverage_rule)

# Restricted of the number of facilities
def facility_count_rule(model):
    return sum(model.x[j] for j in model.J) == model.p
model.FacilityConstraint = Constraint(rule=facility_count_rule)

# Objective function: minimize the total weighted coverage
def obj_rule(model):
    return sum(model.w[i] * model.y[i] for i in model.I)
model.Obj = Objective(rule=obj_rule, sense=minimize)

# ---------------
# Model solution
# ---------------
solver = SolverFactory('glpk')
solver.solve(model)

print("\nVariables x_j:")
for j in model.J:
    print(f"x[{j}] = {model.x[j].value}")

print("\nVariables y_i:")
for i in model.I:
    print(f"y[{i}] = {model.y[i].value}")

print("\nCobertura total =", model.Obj())


Variables x_j:
x[1] = 1.0
x[2] = 0.0
x[3] = 0.0

Variables y_i:
y[1] = 0.0
y[2] = 0.0
y[3] = 1.0
y[4] = 1.0
y[5] = 0.0

Cobertura total = 140.0


# Modification 2: Set selection penalty by JessicaSalazar

A subset with a high penalty value will be more costly to open, and the solver will try not to select it unless it is necessary to cover the demand.

This can mean that some subsets might be saturated and therefore difficult to service, so we penalize them in order not to prioritize them in the solution unless necessary.

In [8]:
# ----------------------
# Set of data
# ----------------------
I_data = [1, 2, 3, 4, 5]
J_data = [1, 2, 3]

# -------------------------------------------
# Dictionaries for each of demand
# -------------------------------------------
N_i_data = {
    1: [1, 2],
    2: [1],
    3: [2, 3],
    4: [2],
    5: [1, 3]
}
w_data = {
    1: 100.0,
    2: 50.0,
    3: 80.0,
    4: 60.0,
    5: 120.0
}

# -----------------------------
# Minimun number of facilities
# -----------------------------
p_value = 1

# ----------------------------------------------
# I define a cost of c[j] for each node j
# (I can modify it to review the changes)
# ----------------------------------------------
penalty = {
    1: 1,   # Open the facility 0, it cost 1
    2: 1,   # Open the facility 1, it cost 1
    3: 4   # Open the facility 2, it cost 4 (selection panlty)
}

# -----------------------------------------------------
# Create model
# The model uded is Maximal Covering Location Problem
# -----------------------------------------------------
model = ConcreteModel(name="MCLP")

# Sets
model.I = Set(initialize=I_data)
model.J = Set(initialize=J_data)

# Parameters
model.w = Param(model.I, initialize=w_data, within=NonNegativeReals)
model.p = Param(initialize=p_value, within=NonNegativeIntegers)

# Variables
model.x = Var(model.J, within=Binary)
model.y = Var(model.I, within=Binary)

# Covering restricted:
# Each node i must be covered by at least one facility in N[i]
def coverage_rule(model, i):
    return model.y[i] + sum(model.x[j] for j in N_i_data[i]) >= 1
model.CoverageConstraint = Constraint(model.I, rule=coverage_rule)

# Restricted of the number of facilities
def facility_count_rule(model):
    return sum(penalty[j] * model.x[j] for j in model.J) == model.p
model.FacilityConstraint = Constraint(rule=facility_count_rule)

# Objective function: minimize the total weighted coverage
def obj_rule(model):
    return sum(model.w[i] * model.y[i] for i in model.I)
model.Obj = Objective(rule=obj_rule, sense=minimize)

# ---------------
# Model solution
# ---------------
solver = SolverFactory('glpk')
solver.solve(model)

print("\nVariables x_j:")
for j in model.J:
    print(f"x[{j}] = {model.x[j].value}")

print("\nVariables y_i:")
for i in model.I:
    print(f"y[{i}] = {model.y[i].value}")

print("\nCobertura total =", model.Obj())


Variables x_j:
x[1] = 1.0
x[2] = 0.0
x[3] = 0.0

Variables y_i:
y[1] = 0.0
y[2] = 0.0
y[3] = 1.0
y[4] = 1.0
y[5] = 0.0

Cobertura total = 140.0
